# Homework #4: Pop Music Transformer Colab notebook

Generate pop piano music using Transformer models. In this task, we use REMI, which stands for `REvamped MIDI-derived events.` 

REMI is a new event representation for converting MIDI scores into text-like discrete tokens. 

- Experience MIDI, REMI data processing
- Experience Transformer architecture


# Ready to start!


Google Drive Mount


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a new working folder

In [ ]:
mkdir 'drive/MyDrive/GCT634_AI613/'

mkdir: cannot create directory ‘drive/MyDrive/GCT634_AI613/’: File exists


Move to your working directory with '%cd' command.

In [ ]:
%cd /content/drive/MyDrive/GCT634_AI613

/content/drive/MyDrive/GCT634_AI613


Clone git repository
https://github.com/juhannam/gct634-ai613-2021/HW4

In [ ]:
!git clone https://github.com/juhannam/gct634-ai613-2021/HW4

In [ ]:
%cd ./HW4/

/content/drive/MyDrive/GCT634_AI613/HW4


Install Dependencies
Check the README for required dependencies and install them.

** Colab already has tensorflow. However, since version 1.14 is required, re-installation may be required.

In [ ]:
!pip install miditoolkit
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14

     |████████████████████████████████| 51 kB 2.8 MB/s 
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 377.1 MB 8.3 kB/s 
     |████████████████████████████████| 488 kB 48.0 MB/s 
     |████████████████████████████████| 3.1 MB 26.2 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:


In [ ]:
import utils
import modules
import model
import chord_recognition
import miditoolkit


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 



---



# Practice 1 : MIDI to Event representation conversion


## Read MIDI


In [ ]:
file_name = 'sample_data/sample_data.midi'
midi_obj = miditoolkit.midi.parser.MidiFile(file_name)

In [ ]:

print('-MIDI META DATA-')
print(midi_obj)
print()
print('-Instrument information(GM number)-')
print(*midi_obj.instruments, sep='\n')

-MIDI META DATA-
ticks per beat: 480
max tick: 288001
tempo changes: 1
time sig: 1
key sig: 0
markers: 2
lyrics: False
instruments: 1

-Instrument information(GM number)-
Instrument(program=0, is_drum=False, name="piano")


Print note events
- show 10 note events of instrument 0
- https://www.inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies

In [ ]:
print(*midi_obj.instruments[0].notes[:10], sep='\n')

Note(start=0, end=710, pitch=43, velocity=34)
Note(start=501, end=723, pitch=55, velocity=23)
Note(start=252, end=741, pitch=50, velocity=34)
Note(start=688, end=955, pitch=58, velocity=36)
Note(start=1182, end=1528, pitch=50, velocity=36)
Note(start=946, end=1629, pitch=43, velocity=29)
Note(start=1468, end=1645, pitch=55, velocity=24)
Note(start=1660, end=2078, pitch=58, velocity=37)
Note(start=16, end=2598, pitch=62, velocity=49)
Note(start=2418, end=2614, pitch=55, velocity=31)


## Convert to REMI events

REMI, which stands for REvamped MIDI-derived events, is a new event representation we propose for converting MIDI scores into text-like discrete tokens. Compared to the MIDI-like event representation adopted in exising Transformer-based music composition models, REMI provides sequence models a metrical context for modeling the rhythmic patterns of music. 

Read midi into "Item"
- Item is container class of events defined in utils.py

In [ ]:
note_items, tempo_items = utils.read_items(file_name)

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=0, end=710, velocity=34, pitch=43)
Item(name=Note, start=16, end=2598, velocity=49, pitch=62)
Item(name=Note, start=252, end=741, velocity=34, pitch=50)
Item(name=Note, start=501, end=723, velocity=23, pitch=55)
Item(name=Note, start=688, end=955, velocity=36, pitch=58)
Item(name=Note, start=946, end=1629, velocity=29, pitch=43)
Item(name=Note, start=1182, end=1528, velocity=36, pitch=50)
Item(name=Note, start=1468, end=1645, velocity=24, pitch=55)
Item(name=Note, start=1660, end=2078, velocity=37, pitch=58)
Item(name=Note, start=1940, end=2634, velocity=34, pitch=41)


In [ ]:
print(*tempo_items[:10], sep='\n')

Item(name=Tempo, start=0, end=None, velocity=None, pitch=64)


Quantize note items

In [ ]:
note_items = utils.quantize_items(note_items, ticks=64) 

In [ ]:
print(*note_items[:10], sep='\n')

Item(name=Note, start=0, end=710, velocity=34, pitch=43)
Item(name=Note, start=0, end=2582, velocity=49, pitch=62)
Item(name=Note, start=256, end=745, velocity=34, pitch=50)
Item(name=Note, start=512, end=734, velocity=23, pitch=55)
Item(name=Note, start=704, end=971, velocity=36, pitch=58)
Item(name=Note, start=960, end=1643, velocity=29, pitch=43)
Item(name=Note, start=1152, end=1498, velocity=36, pitch=50)
Item(name=Note, start=1472, end=1649, velocity=24, pitch=55)
Item(name=Note, start=1664, end=2082, velocity=37, pitch=58)
Item(name=Note, start=1920, end=2614, velocity=34, pitch=41)


extract chord (optional)
- we can use rule-based chord extraction using chord_recognition.py
- chord recognition resolution : 2 beat ~ 4 beat

In [ ]:
chord_items = utils.extract_chords(note_items) 

In [ ]:
print(*chord_items[:10], sep='\n')

Item(name=Chord, start=0, end=1920, velocity=None, pitch=G:min)
Item(name=Chord, start=1920, end=2880, velocity=None, pitch=G:min)
Item(name=Chord, start=2880, end=3840, velocity=None, pitch=D:min)
Item(name=Chord, start=3840, end=5760, velocity=None, pitch=D#:maj)
Item(name=Chord, start=5760, end=7653, velocity=None, pitch=D:min)


group items

In [ ]:
items = chord_items + tempo_items + note_items
max_time = note_items[-1].end
groups = utils.group_items(items, max_time)

In [ ]:
for g in groups:
    print(*g, sep='\n')
    print()

0
Item(name=Chord, start=0, end=1920, velocity=None, pitch=G:min)
Item(name=Tempo, start=0, end=None, velocity=None, pitch=64)
Item(name=Note, start=0, end=710, velocity=34, pitch=43)
Item(name=Note, start=0, end=2582, velocity=49, pitch=62)
Item(name=Note, start=256, end=745, velocity=34, pitch=50)
Item(name=Note, start=512, end=734, velocity=23, pitch=55)
Item(name=Note, start=704, end=971, velocity=36, pitch=58)
Item(name=Note, start=960, end=1643, velocity=29, pitch=43)
Item(name=Note, start=1152, end=1498, velocity=36, pitch=50)
Item(name=Note, start=1472, end=1649, velocity=24, pitch=55)
Item(name=Note, start=1664, end=2082, velocity=37, pitch=58)
1920

1920
Item(name=Chord, start=1920, end=2880, velocity=None, pitch=G:min)
Item(name=Note, start=1920, end=2614, velocity=34, pitch=41)
Item(name=Note, start=2176, end=2727, velocity=32, pitch=50)
Item(name=Note, start=2432, end=2628, velocity=31, pitch=55)
Item(name=Note, start=2624, end=2820, velocity=35, pitch=58)
Item(name=Chord,

In [ ]:
events = utils.item2event(groups)

In [ ]:
print(*events[:30], sep='\n')

Event(name=Bar, time=None, value=None, text=1)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Chord, time=0, value=G:min, text=G:min)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Tempo Class, time=0, value=slow, text=None)
Event(name=Tempo Value, time=0, value=34, text=None)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Note Velocity, time=0, value=8, text=34/32)
Event(name=Note On, time=0, value=43, text=43)
Event(name=Note Duration, time=0, value=11, text=710/720)
Event(name=Position, time=0, value=1/16, text=0)
Event(name=Note Velocity, time=0, value=12, text=49/48)
Event(name=Note On, time=0, value=62, text=62)
Event(name=Note Duration, time=0, value=42, text=2582/2580)
Event(name=Position, time=256, value=3/16, text=256)
Event(name=Note Velocity, time=256, value=8, text=34/32)
Event(name=Note On, time=256, value=50, text=50)
Event(name=Note Duration, time=256, value=7, text=489/480)
Event(name=Position, time=512, value=5/16, text=512)
Even

Save Events as a file

In [ ]:
from collections import Counter
import pickle

pickle.dump(events, open('events.pkl', 'wb'))

Load saved events file



In [ ]:
events = pickle.load(open('events.pkl', 'rb'))

Convert events to MIDI

In [ ]:
midi_path = './converted_MIDI.midi'

utils.write_midi_events(None, None, midi_path, prompt_path=None, events=events)

./converted_MIDI.midi




---

\

# Task 1  : MIDI to Event representation conversion

1. Find your favorite piano songs
2. Transcribe the piano songs using the pre-trained onset and frames model or your own model (HW3)
3. Import your MIDI files, convert it to REMI and convert it back to MIDI file

You can try adjusting the variables. e.g. Modify the quantize_items' parameter

You should submit 

- Transcribed MIDI file
- REMI events.pkl
- Restored MIDI file(quantized)




---

\

# Practice 2: Pop_Music_Transformer

## Download Pre-trained Checkpoints
download two pre-trained checkpoints for generating samples.

* REMI-tempo-checkpoint (428 MB)


In [ ]:
!pip install gdown
!gdown --id 1gxuTSkF51NP04JZgTE46Pg4KQsbHQKGo
!unzip REMI-tempo-checkpoint.zip

Downloading...
From: https://drive.google.com/uc?id=1gxuTSkF51NP04JZgTE46Pg4KQsbHQKGo
To: /content/drive/My Drive/GCT634_AI613/HW4/REMI-tempo-checkpoint.zip
100% 449M/449M [00:15<00:00, 29.8MB/s]
Archive:  REMI-tempo-checkpoint.zip
replace REMI-tempo-checkpoint/checkpoint? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Download MIDI dataset
download MIDI files including local tempo changes and estimated chord. (5 MB) and put in data directory.

* data/train: 775 files used for training models
* data/evaluation: 100 files (prompts) used for the continuation experiments

In [ ]:
!gdown --id 1JUDHGrVYGyHtjkfI2vgR1xb2oU8unlI3
!unzip data.zip

Downloading...
From: https://drive.google.com/uc?id=1JUDHGrVYGyHtjkfI2vgR1xb2oU8unlI3
To: /content/drive/My Drive/GCT634_AI613/HW4/data.zip
100% 4.77M/4.77M [00:00<00:00, 71.4MB/s]
Archive:  data.zip
   creating: data/
   creating: data/evaluation/
  inflating: data/evaluation/000.midi  
  inflating: data/evaluation/001.midi  
  inflating: data/evaluation/002.midi  
  inflating: data/evaluation/003.midi  
  inflating: data/evaluation/004.midi  
  inflating: data/evaluation/005.midi  
  inflating: data/evaluation/006.midi  
  inflating: data/evaluation/007.midi  
  inflating: data/evaluation/008.midi  
  inflating: data/evaluation/009.midi  
  inflating: data/evaluation/010.midi  
  inflating: data/evaluation/011.midi  
  inflating: data/evaluation/012.midi  
  inflating: data/evaluation/013.midi  
  inflating: data/evaluation/014.midi  
  inflating: data/evaluation/015.midi  
  inflating: data/evaluation/016.midi  
  inflating: data/evaluation/017.midi  
  inflating: data/evaluation/01

Create the directories

In [ ]:
mkdir './result/'

mkdir: cannot create directory ‘./result/’: File exists


In [ ]:
mkdir './finetune_train_data/'

mkdir: cannot create directory ‘./finetune_train_data/’: File exists


## Import the required modules

In [ ]:
from model import PopMusicTransformer
from glob import glob
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


## Defines generation functions

You can change various parameters (temperature, top-k..)


What is the function of the inputs "temperature" and "topk"?
It is the temperature-controlled stochastic sampling methods are used for generating text from a trained language model. You can find out more details in the reference paper [CTRL: 4.1 Sampling](https://einstein.ai/presentations/ctrl.pdf).

In [ ]:

def Generation():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-tempo-checkpoint',
        is_training=False)
    
    # generate from scratch
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/from_scratch.midi',
        prompt=None)

    # generate continuation
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/test_continuation.midi',
        prompt='./sample_data/sample_data.midi' 

    )
    
    # close model
    model.close()
    

In [ ]:
Generation()

INFO:tensorflow:Restoring parameters from REMI-tempo-checkpoint/model
./result/from_scratch.midi
./result/test_continuation.midi


## Tensorflow Reset 

The context is maintained in Colab, you need to reset tf before running other codes.

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

## Defines Finetune functions

If you want to make further modifications, you can check the 'model.py'.

In [ ]:
def Finetune():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-tempo-checkpoint',
        is_training=True)
    # prepared data
    midi_paths = glob('finetune_traindata/*.midi') # you need to revise it
    print(midi_paths)
    training_data = model.prepare_data(midi_paths=midi_paths)


    output_checkpoint_folder = 'REMI-finetune' # your decision
    if not os.path.exists(output_checkpoint_folder):
        os.mkdir(output_checkpoint_folder)
    
    # finetune
    model.finetune(
        training_data=training_data,
        output_checkpoint_folder=output_checkpoint_folder,Epoch_n = 1)

    # close
    model.close()

In [ ]:
Finetune()

dictionary.pkl is required for generation based on the Finetune output checkpoint.

In [ ]:
import shutil

shutil.copyfile("./REMI-tempo-checkpoint/dictionary.pkl", "./REMI-finetune/dictionary.pkl")

FileNotFoundError: ignored

## Generation based on finetune checkpoints

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

In [ ]:

def Generation_with_finetune():
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-finetune',
        is_training=False)
    
    # generate from scratch
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/finetune_from_scratch.midi',
        prompt=None)

    # generate continuation
    model.generate(
        n_target_bar=16,
        temperature=1.2,
        topk=5,
        output_path='./result/finetune_continuation.midi',
        prompt='./sample_data/sample_data.midi'  #feel free to change

    )
    
    # close model
    model.close()

In [ ]:
Generation_with_finetune()

# Task 2  : Fine-tune the model with your own MIDI files

1. Find your favorite piano songs
2. Transcribe the piano songs using the pre-trained onset and frames model or your own model (HW3)
3. Using transcription results to fine-tune the Pop Music Transformer pre-trained model

You should submit  

- **MIDI file used for training**
- **Various generation results**
- **Report** about the learning process and subjective evaluatio

FYI, We are not gonna judge the musical quality of the results


## How to synthesize the audio files?

We strongly recommend using DAW and VSTI : [Reaper](https://www.reaper.fm/) (FREE DAW), [Spitfire audio LABS](https://labs.spitfireaudio.com/#type=&search=&new=true) (Free Piano VSTI)  to open/play the generated MIDI files. Or, you can use [FluidSynth](https://github.com/FluidSynth/fluidsynth) with a [SoundFont](https://sites.google.com/site/soundfonts4u/). However, it may not be able to correctly handle the tempo changes (see fluidsynth/issues/141).